<a href="https://colab.research.google.com/github/is0280fp/google_colab/blob/use_image_as_input/training_spectrogram.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# ラベルの読み込み
import pickle
with open("/content/drive/MyDrive/pickle/label_five_class.pickle", mode="rb") as f:
   label = pickle.load(f)
f.close()

In [3]:
# データセット内訳チェック
import numpy as np

print(np.array(np.where(label == 0)).shape)
print(np.array(np.where(label == 1)).shape)
print(np.array(np.where(label == 2)).shape)
print(np.array(np.where(label == 3)).shape)
print(np.array(np.where(label == 4)).shape)

# NaNチェック
print(np.isnan(label).sum())

(1, 7755)
(1, 12748)
(1, 19873)
(1, 21386)
(1, 13970)
0


In [4]:
label.shape

(75732,)

In [7]:
import tensorflow as tf
from tensorflow.keras import optimizers, regularizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, LSTM, InputLayer, Dense, Dropout, Activation, Flatten, concatenate, Conv1D, MaxPooling1D, Input, Reshape, Bidirectional, GlobalAveragePooling2D
from keras.models import Model
from tensorflow.keras.callbacks import TensorBoard, LearningRateScheduler
import time
import math
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import confusion_matrix, classification_report, plot_confusion_matrix
import seaborn as sns
from sklearn.model_selection import train_test_split
import keras
from PIL import Image
import numpy as np

# NAME = "HC-vs-C-LSTM_2class"
# NAME = "HC-vs-A-LSTM_2class"
# NAME = "HC-vs-K-LSTM_2class"
# NAME = "HC-vs-H-LSTM_2class"
# NAME = "HC-vs-GDs-LSTM_2class"
# NAME = "H-vs-K-vs-A-vs-C-LSTM_4class"
#NAME = "HC-vs-H-vs-K-vs-A-vs-C-LSTM_5class"
# NAME = "H-vs-K-vs-A-vs-C-CNN_4class"
NAME = "HC-vs-Patients-CNN_5class"
# NAME = "HC-vs-GDs-CNN_2class"
# NAME = "HC-vs-H-CNN_2class"
# NAME = "HC-vs-K-CNN_2class"
# NAME = "HC-vs-A-CNN_2class"
# NAME = "HC-vs-C-CNN_2class"

#-----------------------------------初期設定-------------------------------------------
# tensorboardのログ
tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))

# 学習打ち切り目安
early_stopping = EarlyStopping(
                        monitor='val_loss',
                        min_delta=0.0,
                        patience=10,
                )

# checkpointの設定
checkpoint = ModelCheckpoint(
                    filepath="saved_model/{}".format(NAME),
                    monitor='val_loss',
                    save_best_only=True,
                    period=1,
                )

# weightの初期化
# initializer = tf.initializers.he_normal()

#-----------------------------------学習モデル作成-------------------------------------------


#-----------------------------------Optimizer-------------------------------------------
# Optimizer: 多クラス分類+Adam
# model.compile(loss='sparse_categorical_crossentropy', 
#               optimizer= optimizers.Adam(lr=0.002, beta_1= 0.9, beta_2= 0.999),
#               metrics=['accuracy'],
#               )

# Optimizer: 多クラス分類+SGD
# model.compile(loss='sparse_categorical_crossentropy',
#               optimizer= optimizers.SGD(lr=5e-2, decay=1e-2),
#               metrics=['accuracy'],
#               )

# Optimizer: 2クラス分類
# model.compile(loss='binary_crossentropy',
#               optimizer= optimizers.SGD(lr=5e-3),
#               metrics=['accuracy'],
#               )

#-----------------------------------summer出力-------------------------------------------
# model.summary()

#-----------------------------------学習-------------------------------------------
# model.fit(X, train_label,
#           batch_size=5,
#           epochs=1000,
#           validation_split=0.2,
#           callbacks=[tensorboard])

# training parameters = kernel_size * (1Dだったらここは1, 2Dだったらここはkernel_size) * output_channels + num_bias
# 216 = 8 * 1 * 24 + 24

In [8]:
# 学習モデル構成
def lenet(INPUT_SHAPE, num_classes):
    """
    http://tecmemo.wpblog.jp/wp-content/uploads/2017/03/dl_lenet-01.png この表を参考に一部活性化関数を変更してLenetを定義
    """
    initializer = tf.initializers.he_normal()
    model = Sequential()
    # フィルターを6枚用意, 小窓のサイズ5×5, paddingによって入力と出力の画像サイズは同じ
    # from IPython.core.debugger import Pdb; Pdb().set_trace()
    model.add(Conv2D(
        64, kernel_size=3, padding="same",
        input_shape=INPUT_SHAPE, activation="relu",
        kernel_initializer=initializer
    ))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(64, kernel_size=3, strides=1, padding="same",
                     activation="relu", kernel_initializer=initializer))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=2))

    model.add(Conv2D(128, kernel_size=3, strides=1, padding="same",
                     activation="relu", kernel_initializer=initializer))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=2))

    model.add(Conv2D(128, kernel_size=3, strides=1, padding="same",
                     activation="relu", kernel_initializer=initializer))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=2))

    model.add(Conv2D(128, kernel_size=3, strides=1, padding="same",
                     activation="relu", kernel_initializer=initializer))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=2))

    model.add(Conv2D(256, kernel_size=3, strides=1, padding="same",
                     activation="relu", kernel_initializer=initializer))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=2))

    model.add(Conv2D(256, kernel_size=3, strides=1, padding="same",
                     activation="relu", kernel_initializer=initializer))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=2))

    model.add(Conv2D(512, kernel_size=3, strides=1, padding="same",
                     activation="relu", kernel_initializer=initializer))
    model.add(GlobalAveragePooling2D())

    # Flatten()はマトリックスを1次元ベクトルに変換する層
    # FCにつなぐために必要
    model.add(Dense(num_classes, kernel_initializer=initializer))
    model.add(Dropout(0.6))
    model.add(Activation("softmax"))
    model.summary()
    return model

In [10]:
# 入力データの加工

def conv_binary(number):
  V_left_filename = '/content/spectrogram_data/V_left/{}.png'.format(number)
  V_right_filename = '/content/spectrogram_data/V_right/{}.png'.format(number)
  V_left_img = Image.open(V_left_filename)
  V_right_img = Image.open(V_right_filename)
  V_left_img = V_left_img.resize((256, 512))
  V_right_img = V_right_img.resize((256, 512))
  # 色の変換も簡単ですが、できる色が制限されます。
  # print(img.mode) # RGBA
  V_left_rgb = np.array(V_left_img.convert('RGB'))
  V_right_rgb = np.array(V_right_img.convert('RGB'))
  # numpy配列の取得
  img_array = np.concatenate([V_left_rgb, V_right_rgb], axis=1)
  return img_array

In [55]:
def get_batch(batch_size, X, Y):
    """
    batchを取得する関数
    """
    SIZE = len(X)
    # n_batchs
    n_batchs = SIZE//batch_size
    # for でyield
    i = 0
    while ((i+batch_size) < SIZE):
        # print("doing", i, "/", n_batchs)
        # Y_batch = Y[(i * n_batchs):(i * n_batchs + batch_size)]
        Y_batch = Y[i:(i + batch_size)]
        
        #あるbatchのfilenameの配列を持っておく
        # X_batch_name = X[(i * n_batchs):(i * n_batchs + batch_size)]
        X_batch_name = X[i:(i + batch_size)]

        # filenameにしたがってバッチのtensorを構築
        X_batch = np.array([conv_binary(file) for file in X_batch_name])
        
        # これで(batch_size, 28, 28, 1)のtrainのテンソルが作られる
        i += 1
        yield X_batch, Y_batch

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from keras.callbacks import LearningRateScheduler

# def step_decay(epoch):
#     x = 0.1
#     if epoch >= 150: x = 0.01
#     if epoch >= 225: x = 0.001
#     return x
# lr_decay = LearningRateScheduler(step_decay)

model = lenet((512, 512, 3), 5)
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=optimizers.Adam(lr=0.001, beta_1= 0.9, beta_2= 0.999),
              metrics=['accuracy'])
N_EPOCHS = 1000
BATCH_SIZE = 32

# acquire the .png name
TRAINS = np.array(range(len(label)))
y = label
# # split test and train
# X_train, X_test, Y_train, Y_test = train_test_split(
#     TRAINS, y, test_size=0.1, random_state=42)

# split test and the others
X_train_and_val, X_test, Y_train_and_val, Y_test = train_test_split(
    TRAINS, y, test_size=0.1, random_state=42)

# split k-folds (その中の一つのfoldがvalidationのためのtestデータとなる)
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
for X_index, Y_index in skf.split(X_train_and_val, Y_train_and_val):
    # print("train_index:", train_index, "val_index:", val_index)
    # print("train data", X_train_and_val[X_index])
    # print("train label", Y_train_and_val[X_index])
    # print("test data", X_train_and_val[Y_index])
    # print("test label", Y_train_and_val[Y_index])

    train_data = X_train_and_val[X_index]
    train_label = Y_train_and_val[X_index]
    test_data = X_train_and_val[Y_index]
    test_label = Y_train_and_val[Y_index]

    steps_per_epoch=len(X_train_and_val[X_index])//BATCH_SIZE 
    validation_steps=len(X_train_and_val[Y_index])//BATCH_SIZE 

training_generator = get_batch(BATCH_SIZE, train_data, train_label)
validation_generator = get_batch(BATCH_SIZE, test_data, test_label)

# from IPython.core.debugger import Pdb; Pdb().set_trace()
model.fit_generator(
    generator=training_generator,
    validation_data=validation_generator,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps,
    epochs=N_EPOCHS
    # callbacks=[tensorboard]
)

In [20]:
print(steps_per_epoch)
print(len(X_train_and_val[X_index]))
print(BATCH_SIZE)

7667
61343
8


In [64]:
print(X_train_and_val[X_index].shape)
print(X_index.shape)
print(X_train_and_val[X_index][0])
print(X_index[0])
TRAINS

(61343,)
(61343,)
10137
0


array([    0,     1,     2, ..., 75729, 75730, 75731])

In [ ]:
# データの読み込み 
# yesを選ぶように!
!unzip /content/drive/MyDrive/zip_spectrogram_data/AP_left.zip
!unzip /content/drive/MyDrive/zip_spectrogram_data/AP_right.zip
!unzip /content/drive/MyDrive/zip_spectrogram_data/AP_right_69826.zip

In [ ]:
# データの読み込み
# yesを選ぶように!
!unzip /content/drive/MyDrive/zip_spectrogram_data/ML_left.zip
!unzip /content/drive/MyDrive/zip_spectrogram_data/ML_left_58106.zip
!unzip /content/drive/MyDrive/zip_spectrogram_data/ML_right.zip

In [ ]:
# データの読み込み
# yesを選ぶように!
!unzip /content/drive/MyDrive/zip_spectrogram_data/V_left.zip
!unzip /content/drive/MyDrive/zip_spectrogram_data/V_left_48327.zip
!unzip /content/drive/MyDrive/zip_spectrogram_data/V_right.zip
!unzip /content/drive/MyDrive/zip_spectrogram_data/V_right_34533.zip